In [21]:
!pip install -q crewai crewai.tools

/opt/anaconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=91114) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


In [23]:
!pip install -q yfinance

In [25]:
import os 

os.environ["OPENAI_API_KEY"] = ""

In [27]:
import yfinance as yf
from pydantic import BaseModel, ValidationError, Field
from crewai.tools import tool

# from crewai_tools import tool

# ticker = yf.Ticker("GME")
# ticker.news
# ticker.dividends
# ticker.history(period="1mo")
# ticker.balance_sheet

@tool("Stock News")
def stock_news(ticker):
    """
    Useful to get news about a stock.
    The input should be a ticker, for example AAPL, NET.
    """
    ticker = yf.Ticker(ticker)
    return ticker.news

@tool("Stock Price")
def stock_price(ticker):
    """
    Useful to get stock price data.
    The input should be a ticker, for example AAPL, NET.
    """
    ticker = yf.Ticker(ticker)
    return ticker.history(period='1mo')

@tool("Income Statement")
def income_stmt(ticker):
    """
    Useful to get the income statement of a company.
    The input to this tool should be a ticker, for example AAPL, NET.
    """
    ticker = yf.Ticker(ticker)
    return ticker.income_stmt

@tool("Balance Sheet")
def balance_sheet(ticker):
    """
    Useful to get the balance sheet of a company.
    The input to this tool should be a ticker, for example AAPL, NET.
    """
    ticker = yf.Ticker(ticker)
    return ticker.balance_sheet

@tool("Insider Transactions")
def insider_transactions(ticker):
    """
    Useful to get insider transactions of a company.
    The input to this tool should be a ticker, for example AAPL, NET.
    """
    ticker = yf.Ticker(ticker)
    return ticker.insider_transactions
    
    

In [29]:
from crewai import Agent
from crewai_tools import ScrapeWebsiteTool

scrape_tool = ScrapeWebsiteTool()


researcher = Agent(
    role="Researcher",
    goal="Gather and interpret vast amounts of data to provide a comprehensive overview of the sentiment and news surrounding a stock",
    backstory="You're skilled in gathering and interpreting data from various sources. You read each data source carefully and extract the most important information. Your insights are crucial for making informed investment decisions.",
    tools=[
        scrape_tool, 
        stock_news
    ],
    verbose=True
)
technical_analyst = Agent(
    role="Technical Analyst",
    goal="Analyze the movements of a stock and provide insights on trends, entry points, resistance and support levels",
    backstory="An expert in techinical analysis, you're known for your ability to predict stock prices. You provide valuable insights to your customers.",
    tools=[
        stock_price
    ],
    verbose=True
)
financial_analyst = Agent(
    role="Financial Analyst",
    goal="Use financial statements, insider trading data and other metrics to evaluate a stock's financial health and performance.",
    backstory="You're a very experienced investment advisor that looks at a company's financial health, market sentiment, and qualitative data to make informed recommendations.",
    tools=[
        income_stmt,
        balance_sheet,
        insider_transactions
    ],
    verbose=True
)
hedge_fund_manager = Agent(
    role="Hedge Fund Manager",
    goal="Manage a portfolio of stocks and make investment decisions to maximize returns using insights from financial analysts and researchers",
    backstory="You're a seasoned hedge fund manager with a proven track record of making profitable investments. You always impress your clients.",
    verbose=True
)

In [31]:
from crewai import Task

research = Task(
    description="Gather and analyze the latest news and market sentiment surrounding {ticker}'s stock. Provide a summary of the news and any notable shifts in sentiment.",
    agent=researcher,
    expected_output="Your final answer MUST be a detailed summary of the news and market sentiment surrounding the stock.",
)
technical_analysis = Task(
    description="Conduct a technical analysis of the {ticker} stock price movements and identify kew support and resistance levels chart patterns.",
    agent=technical_analyst,
    expected_output="Your final answer MUST be a report with potential entry points, price targets and any other relevant information.",
)
financial_analysis = Task(
    description="Analyze the {ticker}'s financial statements, balance sheet, insider trading data and other metrics to evaluate {ticker}'s financial health and performance.",
    agent=financial_analyst,
    expected_output="Your final answer MUST be a report with an overview of {ticker}'s revenue, earnings, cash flow, and other key financial metrics.",
)
investment_recommendation = Task(
    description="Based on the research, technical analysis, and financial analysis reports, provide a detailed investment recommendation for {ticker} stock.",
    agent=hedge_fund_manager,
    expected_output="Your final answer MUST be a detailed recommendation to BUY, SELL, or HOLD the stock. Provide a clear rationale for your recommendation.",
    context =  [
        research, 
        technical_analysis, 
        financial_analysis,
    ],
    output_file="investment_recommendation.md"
)

In [35]:
from crewai import Crew

crew = Crew(
    tasks=[
        research,
        technical_analysis,
        financial_analysis,
        investment_recommendation
    ],
    agents=[
        researcher,
        technical_analyst,
        financial_analyst,
        hedge_fund_manager
    ],
    verbose=True
)

result = crew.kickoff(
    inputs={
        "company":"Tesla",
        "ticker": "TSLA"
    }
)

"""
Analyze the Tesla's financial statements, balance sheet, insider trading data and other metrics to evaluate {company}'s financial health and performance
"""

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 519ab399-acc6-44ee-8b54-b2e3968bb2d4                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Researcher                                                                                              │
│                                                                                                                 │
│  Task: Gather and analyze the latest news and market sentiment surrounding TSLA's stock. Provide a summary of   │
│  the news and any notable shifts in sentiment.                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/opt/anaconda3/lib/python3.12/site-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

/opt/anaconda3/lib/python3.12/site-packages/ipywidgets/widgets/widget_output.py:112: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  self.msg_id = ip.kernel._parent_header['header']['msg_id']

...

2025-07-07 09:33:59,099 - 13014953984 - telemetry.py-telemetry:51 - ERROR: HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x34f1daf90>, 'Connection to telemetry.crewai.com timed out. (connect timeout=30)'))


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Hedge Fund Manager                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  After analyzing the current market conditions, technical indicators, and financial health of Tesla Inc.        │
│  (TSLA), I recommend a cautious **HOLD** on the stock at this moment. Here are the detailed reasons and         │
│  insights behind this recommendation:                                                                           │
│                                                                                                                 │
│  1. **Market Sentiment:**                                                                                       │
│     The recent 7.35% decline in TSLA's stock price to $292.18 reflects a significant shift in investor          │
│  sentiment, largely due to external factors such as a downgrade from William Blair and mounting concerns over   │
│  competitive pressures in the electric vehicle sector. Analysts are re-evaluating their positions, which is     │
│  indicative of uncertain growth prospects ahead.                                                                │
│                                                                                                                 │
│  2. **Technical Analysis:**                                                                                     │
│     - **Current Price Points**: As TSLA trades at $292.18, it is positioned near an immediate support level     │
│  around $290. However, the volatility noted in its 52-week range (low of $182.00, high of $488.54) signifies    │
│  potential risks.                                                                                               │
│     - **Support and Resistance Levels**: Key support is identified at $290 and further below at $282.           │
│  Resistance levels are observed at $305 and $320, where selling pressure emerges. If TSLA can reclaim the $305  │
│  mark and show momentum, it may indicate a reversal.                                                            │
│     - The bearish trend established by the recent downgrade suggests caution in entering a long position until  │
│  clearer signals emerge.                                                                                        │
│                                                                                                                 │
│  3. **Financial Health Insights:**                                                                              │
│     - **Revenue Growth**: While Tesla has historically demonstrated robust revenue growth, looming competitive  │
│  threats are putting pressure on profit margins. Analyst downgrades highlight skepticism regarding future       │
│  profitability.                                                                                                 │
│     - **Balance Sheet Strength**: Tesla maintains a strong asset base but rising liabilities or diminished      │
│  equity levels could alter financial stability, warranting a risk-managed approach for potential investors.     │
│     - **Insider Sentiment**: Caution from insiders, reflected in lower levels of buying amidst current stock    │
│  performance, adds to the cautious outlook.                                                                     │
│                                                                                                                 │
│  4. **Strategic Implications:**                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 67f6f1e8-55c8-454d-9f14-3a66997e3bc6                                                                     │
│  Agent: Hedge Fund Manager                                                                                      │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 519ab399-acc6-44ee-8b54-b2e3968bb2d4                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: After analyzing the current market conditions, technical indicators, and financial health of     │
│  Tesla Inc. (TSLA), I recommend a cautious **HOLD** on the stock at this moment. Here are the detailed reasons  │
│  and insights behind this recommendation:                                                                       │
│                                                                                                                 │
│  1. **Market Sentiment:**                                                                                       │
│     The recent 7.35% decline in TSLA's stock price to $292.18 reflects a significant shift in investor          │
│  sentiment, largely due to external factors such as a downgrade from William Blair and mounting concerns over   │
│  competitive pressures in the electric vehicle sector. Analysts are re-evaluating their positions, which is     │
│  indicative of uncertain growth prospects ahead.                                                                │
│                                                                                                                 │
│  2. **Technical Analysis:**                                                                                     │
│     - **Current Price Points**: As TSLA trades at $292.18, it is positioned near an immediate support level     │
│  around $290. However, the volatility noted in its 52-week range (low of $182.00, high of $488.54) signifies    │
│  potential risks.                                                                                               │
│     - **Support and Resistance Levels**: Key support is identified at $290 and further below at $282.           │
│  Resistance levels are observed at $305 and $320, where selling pressure emerges. If TSLA can reclaim the $305  │
│  mark and show momentum, it may indicate a reversal.                                                            │
│     - The bearish trend established by the recent downgrade suggests caution in entering a long position until  │
│  clearer signals emerge.                                                                                        │
│                                                                                                                 │
│  3. **Financial Health Insights:**                                                                              │
│     - **Revenue Growth**: While Tesla has historically demonstrated robust revenue growth, looming competitive  │
│  threats are putting pressure on profit margins. Analyst downgrades highlight skepticism regarding future       │
│  profitability.                                                                                                 │
│     - **Balance Sheet Strength**: Tesla maintains a strong asset base but rising liabilities or diminished      │
│  equity levels could alter financial stability, warranting a risk-managed approach for potential investors.     │
│     - **Insider Sentiment**: Caution from insiders, reflected in lower levels of buying amidst current stock    │
│  performance, adds to the cautious outlook.                                                                     │
│                                                       

"\nAnalyze the Tesla's financial statements, balance sheet, insider trading data and other metrics to evaluate {company}'s financial health and performance\n"